In [ ]:
!pip install datasets litellm honeyhive

In [ ]:
from honeyhive import HoneyHiveTracer

# Add this code at the beginning of your AI pipeline code
HoneyHiveTracer.init(
    api_key="dXV3cXpoZmFwb3NsY3N4N3lidmE2aQ==",
    project="text2sql-evals",
)

In [ ]:
import duckdb
from datasets import load_dataset

data = load_dataset("suzyanil/nba-data")["train"]

conn = duckdb.connect(database=":memory:", read_only=False)
conn.register("nba", data.to_pandas())

conn.query("SELECT * FROM nba LIMIT 5").to_df().to_dict(orient="records")[0]

In [ ]:
import os
import json
import duckdb
import pandas as pd
import logging
import re
from datasets import load_dataset
from litellm import completion
from honeyhive import evaluate, evaluator

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set API keys
os.environ["OPENAI_API_KEY"] = "your openai key"
os.environ["GEMINI_API_KEY"] = "your gemini key"
os.environ["ANTHROPIC_API_KEY"] = "your anthropic key"

# Load NBA dataset
data = load_dataset("suzyanil/nba-data")["train"]
conn = duckdb.connect(database=":memory:", read_only=False)
conn.register("nba", data.to_pandas())
logger.info("NBA dataset loaded and registered with DuckDB")

# Get column information
columns = conn.query("DESCRIBE nba").to_df().to_dict(orient="records")
samples = conn.query("SELECT * FROM nba LIMIT 1").to_df().to_dict(orient="records")[0]
sample_rows = "\n".join(
    f"{column['column_name']} | {column['column_type']} | {samples[column['column_name']]}"
    for column in columns
)

# Define system prompt with examples
system_prompt = (
    "You are a SQL expert, and you are given a single table named nba with the following columns:\n\n"
    "Column | Type | Example\n"
    "-------|------|--------\n"
    f"{sample_rows}\n"
    "\n"
    "Write a DuckDB SQL query corresponding to the user's request. "
    "Return just the query text, with no formatting (backticks, markdown, etc.)."
)

# Helper function to clean SQL query from markdown formatting
def clean_sql_query(query):
    """Remove markdown formatting from SQL query"""
    # Remove ```sql and ``` markers
    query = re.sub(r'```sql\s*', '', query)
    query = re.sub(r'```\s*', '', query)
    return query.strip()

# Generate SQL query with OpenAI
def generate_query_openai(question):
    """Generate SQL query from natural language question using OpenAI"""
    logger.info(f"Generating SQL query with OpenAI for question: {question}")
    response = completion(
        model="gpt-4o",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": question,
            },
        ],
    )
    query = response.choices[0].message.content
    query = clean_sql_query(query)
    logger.info(f"Generated query with OpenAI: {query}")
    return query

# Generate SQL query with Gemini
def generate_query_gemini(question):
    """Generate SQL query from natural language question using Gemini"""
    logger.info(f"Generating SQL query with Gemini for question: {question}")
    response = completion(
        model="gemini/gemini-2.0-flash",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt + "\n\nIMPORTANT: Do not include any markdown formatting, code blocks, or backticks in your response. Just provide the raw SQL query.",
            },
            {
                "role": "user",
                "content": question,
            },
        ],
    )
    query = response.choices[0].message.content
    query = clean_sql_query(query)
    logger.info(f"Generated query with Gemini: {query}")
    return query

# Generate SQL query with Anthropic
def generate_query_anthropic(question):
    """Generate SQL query from natural language question using Anthropic"""
    logger.info(f"Generating SQL query with Anthropic for question: {question}")
    response = completion(
        model="claude-3-7-sonnet-20250219",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt + "\n\nIMPORTANT: Do not include any markdown formatting, code blocks, or backticks in your response. Just provide the raw SQL query.",
            },
            {
                "role": "user",
                "content": question,
            },
        ],
    )
    query = response.choices[0].message.content
    query = clean_sql_query(query)
    logger.info(f"Generated query with Anthropic: {query}")
    return query

# Execute SQL query
def execute_query(query):
    """Execute SQL query and return results or error"""
    logger.info(f"Executing query: {query}")
    try:
        results = conn.query(query).fetchdf().to_dict(orient="records")
        logger.info(f"Query executed successfully. Result count: {len(results)}")
        return {"results": results, "error": None}
    except duckdb.Error as e:
        error_msg = str(e)
        logger.error(f"Query execution failed: {error_msg}")
        return {"results": None, "error": error_msg}

# Main text2sql function for OpenAI evaluation
def text2sql_openai(inputs, ground_truths=None):
    """
    Main function that processes a question and returns SQL results using OpenAI.

    Parameters:
      - inputs: Dict with a 'question' key
      - ground_truths: Optional ground truth data (not used in this example)
    """
    logger.info(f"text2sql_openai called with inputs: {inputs}, type: {type(inputs)}")

    # Extract the question from inputs
    if isinstance(inputs, dict) and 'question' in inputs:
        question = inputs['question']
        logger.info(f"Processing question from inputs dict with OpenAI: {question}")
    else:
        error_msg = f"Invalid input format: {inputs}, type: {type(inputs)}"
        logger.error(error_msg)
        return {
            "model": "gpt-4o",
            "query": "",
            "results": [],
            "error": "Invalid input format - expected dict with 'question' key"
        }

    # Generate SQL query
    query = generate_query_openai(question)

    # Execute query
    execution_result = execute_query(query)

    # Combine results
    result = {
        "model": "gpt-4o",
        "query": query,
        "results": execution_result["results"],
        "error": execution_result["error"]
    }

    logger.info(f"text2sql_openai completed. Result: {json.dumps(result, default=str)[:200]}...")
    return result

# Main text2sql function for Gemini evaluation
def text2sql_gemini(inputs, ground_truths=None):
    """
    Main function that processes a question and returns SQL results using Gemini.

    Parameters:
      - inputs: Dict with a 'question' key
      - ground_truths: Optional ground truth data (not used in this example)
    """
    logger.info(f"text2sql_gemini called with inputs: {inputs}, type: {type(inputs)}")

    # Extract the question from inputs
    if isinstance(inputs, dict) and 'question' in inputs:
        question = inputs['question']
        logger.info(f"Processing question from inputs dict with Gemini: {question}")
    else:
        error_msg = f"Invalid input format: {inputs}, type: {type(inputs)}"
        logger.error(error_msg)
        return {
            "model": "gemini-2.0-flash",
            "query": "",
            "results": [],
            "error": "Invalid input format - expected dict with 'question' key"
        }

    # Generate SQL query
    query = generate_query_gemini(question)

    # Execute query
    execution_result = execute_query(query)

    # Combine results
    result = {
        "model": "gemini-2.0-flash",
        "query": query,
        "results": execution_result["results"],
        "error": execution_result["error"]
    }

    logger.info(f"text2sql_gemini completed. Result: {json.dumps(result, default=str)[:200]}...")
    return result

# Main text2sql function for Anthropic evaluation
def text2sql_anthropic(inputs, ground_truths=None):
    """
    Main function that processes a question and returns SQL results using Anthropic.

    Parameters:
      - inputs: Dict with a 'question' key
      - ground_truths: Optional ground truth data (not used in this example)
    """
    logger.info(f"text2sql_anthropic called with inputs: {inputs}, type: {type(inputs)}")

    # Extract the question from inputs
    if isinstance(inputs, dict) and 'question' in inputs:
        question = inputs['question']
        logger.info(f"Processing question from inputs dict with Anthropic: {question}")
    else:
        error_msg = f"Invalid input format: {inputs}, type: {type(inputs)}"
        logger.error(error_msg)
        return {
            "model": "claude-3-7-sonnet",
            "query": "",
            "results": [],
            "error": "Invalid input format - expected dict with 'question' key"
        }

    # Generate SQL query
    query = generate_query_anthropic(question)

    # Execute query
    execution_result = execute_query(query)

    # Combine results
    result = {
        "model": "claude-3-7-sonnet",
        "query": query,
        "results": execution_result["results"],
        "error": execution_result["error"]
    }

    logger.info(f"text2sql_anthropic completed. Result: {json.dumps(result, default=str)[:200]}...")
    return result

# Evaluation functions
@evaluator()
def no_error_evaluator(outputs, inputs, ground_truths=None):
    """Evaluate if the SQL query executed without errors"""
    logger.info(f"no_error_evaluator called with outputs type: {type(outputs)}")

    # Default values for invalid outputs
    if not outputs or not isinstance(outputs, dict):
        return {"score": 0.0, "explanation": "Invalid output format"}

    # Check for error
    error = outputs.get("error")
    model = outputs.get("model", "unknown")
    score = 1.0 if error is None else 0.0
    explanation = f"[{model}] Query executed successfully" if score == 1.0 else f"[{model}] Query execution failed: {error}"

    return {"score": score, "explanation": explanation}

@evaluator()
def has_results_evaluator(outputs, inputs, ground_truths=None):
    """Evaluate if the query returned results"""
    logger.info(f"has_results_evaluator called with outputs type: {type(outputs)}")

    # Default values for invalid outputs
    if not outputs or not isinstance(outputs, dict):
        return {"score": 0.0, "explanation": "Invalid output format"}

    # Check for results
    results = outputs.get("results", [])
    model = outputs.get("model", "unknown")
    has_results = results is not None and len(results) > 0
    score = 1.0 if has_results else 0.0
    explanation = f"[{model}] Query returned {len(results) if results else 0} results" if has_results else f"[{model}] Query returned no results"

    return {"score": score, "explanation": explanation}

@evaluator()
def is_valid_sql_evaluator(outputs, inputs, ground_truths=None):
    """Evaluate if the generated SQL is valid using LLM"""
    logger.info(f"is_valid_sql_evaluator called with outputs type: {type(outputs)}")

    # Default values for invalid outputs
    if not outputs or not isinstance(outputs, dict):
        return {"score": 0.0, "explanation": "Invalid output format"}

    # Get query
    query = outputs.get("query", "")
    model = outputs.get("model", "unknown")
    if not query:
        return {"score": 0.0, "explanation": f"[{model}] No SQL query was generated"}

    IS_SQL_EVAL_TEMPLATE = """You are a SQL expert, is the following a valid SQL query that executes without errors? Return the single word "valid" if it is valid, and "invalid" if it is not.

    [BEGIN SQL QUERY]
    {0}
    [END SQL QUERY]
    """

    response = completion(
        model="gpt-4o",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a SQL expert evaluating SQL queries."
            },
            {
                "role": "user",
                "content": IS_SQL_EVAL_TEMPLATE.format(query)
            }
        ]
    )

    result = response.choices[0].message.content.strip().lower()
    is_valid = "valid" in result
    score = 1.0 if is_valid else 0.0
    explanation = f"[{model}] SQL query is {'valid' if is_valid else 'invalid'}"

    return {"score": score, "explanation": explanation}

# For individual runs without tracing
def run_individual_evaluations():
    # Sample questions
    questions = [
        "Which team won the most games?",
        "Which team won the most games in 2015?",
        "Who led the league in 3 point shots?",
        "Which team had the biggest difference in records across two consecutive years?",
        "What is the average number of free throws per year?",
    ]

    logger.info("Starting individual evaluations")

    # Process each question with all models
    for i, question in enumerate(questions):
        # Create proper input format for the function
        input_data = {"question": question}

        # Process with OpenAI
        result_openai = text2sql_openai(input_data)

        # Process with Gemini
        result_gemini = text2sql_gemini(input_data)

        # Process with Anthropic
        result_anthropic = text2sql_anthropic(input_data)

        # Log results
        print(f"Question: {question}")
        print("\n--- OpenAI (GPT-4o) Results ---")
        print(f"Query: {result_openai['query']}")
        print(f"Error: {result_openai['error']}")
        if result_openai['results']:
            print(f"Results: {result_openai['results'][:2]}...")  # Show first 2 results
        else:
            print("No results")

        print("\n--- Gemini Results ---")
        print(f"Query: {result_gemini['query']}")
        print(f"Error: {result_gemini['error']}")
        if result_gemini['results']:
            print(f"Results: {result_gemini['results'][:2]}...")  # Show first 2 results
        else:
            print("No results")

        print("\n--- Anthropic (Claude) Results ---")
        print(f"Query: {result_anthropic['query']}")
        print(f"Error: {result_anthropic['error']}")
        if result_anthropic['results']:
            print(f"Results: {result_anthropic['results'][:2]}...")  # Show first 2 results
        else:
            print("No results")

        print("-" * 80)
        print(f"Test case {i} complete")

    logger.info("Individual evaluations completed")

# Main execution
if __name__ == "__main__":
    logger.info("Script started")

    # Run individual evaluations without tracing
    run_individual_evaluations()

    # For batch evaluation with the evaluate function
    # Create dataset with proper format following HoneyHive's expected structure
    questions = [
        "Which team won the most games?",
        "Which team won the most games in 2015?",
        "Who led the league in 3 point shots?",
        "Which team had the biggest difference in records across two consecutive years?",
        "What is the average number of free throws per year?",
    ]

    # Create dataset in the EXACT format expected by HoneyHive
    dataset = [
        {
            "inputs": {"question": q},
            "ground_truths": {}  # Optional, can be empty
        }
        for q in questions
    ]

    logger.info(f"Created dataset for evaluation with proper structure: {dataset}")

    # Run evaluation with all models
    logger.info("Starting batch evaluation with OpenAI")
    evaluate(
        function=text2sql_openai,
        hh_api_key="your honeyhive api key",
        hh_project="your honeyhive project name",
        name="your evaluation name",
        dataset=dataset,
        evaluators=[
            no_error_evaluator,
            has_results_evaluator,
            is_valid_sql_evaluator
        ]
    )

    logger.info("Starting batch evaluation with Gemini")
    evaluate(
        function=text2sql_gemini,
        hh_api_key="your honeyhive api key",
        hh_project="your honeyhive project name",
        name="your evaluation name",
        dataset=dataset,
        evaluators=[
            no_error_evaluator,
            has_results_evaluator,
            is_valid_sql_evaluator
        ]
    )

    logger.info("Starting batch evaluation with Anthropic")
    evaluate(
        function=text2sql_anthropic,
        hh_api_key="your honeyhive api key",
        hh_project="your honeyhive project name",
        name="name of your evaluation",
        dataset=dataset,
        evaluators=[
            no_error_evaluator,
            has_results_evaluator,
            is_valid_sql_evaluator
        ]
    )

    logger.info("All Text2SQL evaluations completed and pushed to HoneyHive")
    print("All Text2SQL evaluations completed and pushed to HoneyHive.")